# CORUM

Author: Moshe Silverstein <br/>
Date: 8-17 <br/>
Data Source: http://mips.helmholtz-muenchen.de/corum/#download

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import my_functions as mf
%matplotlib inline

In [2]:
importlib.reload(mf)

<module 'my_functions' from '/Users/moshesilverstein/Documents/Harmonizome/CORUM/my_functions.py'>

# Load Data

In [4]:
df = pd.read_csv('Input/allComplexes.txt', sep='\t')

In [5]:
df.head()

,ComplexID,ComplexName,Organism,Synonyms,Cell line,subunits(UniProt IDs),subunits(Entrez IDs),Protein complex purification method,GO ID,GO description,FunCat ID,FunCat description,Complex comment,PubMed ID,Subunits comment,subunits(Protein name),subunits(Gene name),subunits(Gene name syn),SWISSPROT organism,Disease comment
0,1,BCL6-HDAC4 complex,Human,None,None,P41182;P56524,604;9759,MI:0007- anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0006473;GO:0006476;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;14.07.04;42.10.03;4...,DNA conformation modification (e.g. chromatin)...,Transcriptional repression by BCL6 is thought ...,11929873,None,B-cell lymphoma 6 protein ;Histone deacetylase 4,BCL6;HDAC4,BCL5 LAZ3 ZBTB27 ZNF51;KIAA0288,Homo sapiens (Human);Homo sapiens (Human),None
1,2,BCL6-HDAC5 complex,Human,None,None,P41182;Q9UQL6,604;10014,MI:0007- anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0006473;GO:0006476;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;14.07.04;42.10.03;4...,DNA conformation modification (e.g. chromatin)...,Transcriptional repression by BCL6 is thought ...,11929873,None,B-cell lymphoma 6 protein ;Histone deacetylase 5,BCL6;HDAC5,BCL5 LAZ3 ZBTB27 ZNF51;KIAA0600,Homo sapiens (Human);Homo sapiens (Human),None
2,3,BCL6-HDAC7 complex,Human,None,None,P41182;Q8WUI4,604;51564,MI:0007- anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0006473;GO:0006476;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;14.07.04;42.10.03;4...,DNA conformation modification (e.g. chromatin)...,Transcriptional repression by BCL6 is thought ...,11929873,None,B-cell lymphoma 6 protein ;Histone deacetylase 7,BCL6;HDAC7,BCL5 LAZ3 ZBTB27 ZNF51;HDAC7A,Homo sapiens (Human);Homo sapiens (Human),None
3,4,Multisubunit ACTR coactivator complex,Human,None,None,Q09472;Q92793;Q92831;Q9Y6Q9,2033;1387;8850;8202,MI:0004-affinity chromatography technologies;M...,GO:0045893;GO:0023052;GO:0005634,"positive regulation of transcription, DNA-temp...",11.02.03.04.01;30.01;70.10,transcription activation;cellular signalling;n...,Cofactor ACTR binds directly nuclear receptors...,9267036,None,Histone acetyltransferase p300;CREB-binding pr...,EP300;CREBBP;KAT2B;NCOA3,"P300;CBP;PCAF;AIB1, BHLHE42, RAC3, TRAM1, ACTR",Homo sapiens (Human);Homo sapiens (Human);Homo...,None
4,9,6S-nuclear aryl hydrocarbon (Ah) receptor liga...,Mouse,None,None,P30561;P53762,11622;11863,MI:0019- coimmunoprecipitation,GO:2001141;GO:0006355;GO:0003677;GO:0023052;GO...,regulation of RNA biosynthetic process;regulat...,11.02.03.04;16.03.01;30.01;34.11.03;70.10,transcriptional control;DNA binding;cellular s...,"Arnt contains a basic helix-loop-helix motif, ...",1317062,None,Aryl hydrocarbon receptor ;Aryl hydrocarbon re...,Ahr;Arnt,;None,Mus musculus (Mouse);Mus musculus (Mouse),None


In [6]:
df.shape

(3633, 20)

# Get Relevant Data 

In [7]:
df['Organism'].unique()

array(['Human', 'Mouse', 'Pig', 'Bovine', 'Rat', 'Mammalia', 'Rabbit',
       'Dog', 'Hamster', 'MINK'], dtype=object)

In [8]:
# Get only disered organisims
human = df[df['Organism'] == 'Human'].copy()
mouse = df[df['Organism'] == 'Mouse'].copy()
rat = df[df['Organism'] == 'Rat'].copy()

df = pd.concat([human, mouse])
df = pd.concat([df, rat])

In [11]:
df = df[['ComplexName', 'subunits(Gene name)']]

In [12]:
df.head()

,ComplexName,subunits(Gene name)
0,BCL6-HDAC4 complex,BCL6;HDAC4
1,BCL6-HDAC5 complex,BCL6;HDAC5
2,BCL6-HDAC7 complex,BCL6;HDAC7
3,Multisubunit ACTR coactivator complex,EP300;CREBBP;KAT2B;NCOA3
5,Condensin I complex,SMC2;NCAPH;NCAPD2;NCAPG;SMC4


In [10]:
df.shape 

(3332, 20)

In [35]:
df_interactions = pd.DataFrame()

for i, index in enumerate(df.index):
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progeres: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    
    
    lst2 = df.ix[index, 'subunits(Gene name)'].split(';')
    lst1 = [df.ix[index, 'ComplexName'].split('(')[0]]*(len(lst2))
    temp = pd.DataFrame()
    temp['GeneSymbol'] = lst2
    temp['Complex'] = lst1
    df_interactions = pd.concat([df_interactions, temp]) 

In [36]:
df_interactions.head()

,GeneSymbol,Complex
0,BCL6,BCL6-HDAC4 complex
1,HDAC4,BCL6-HDAC4 complex
0,BCL6,BCL6-HDAC5 complex
1,HDAC5,BCL6-HDAC5 complex
0,BCL6,BCL6-HDAC7 complex


# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [37]:
df_interactions.set_index('GeneSymbol', inplace=True)

In [38]:
mf.mapgenesymbols(df_interactions)

# Drop Duplicates

In [39]:
df_interactions.reset_index(inplace=True)

In [40]:
df_interactions.drop_duplicates(inplace=True)

In [41]:
df_interactions.shape

(11282, 2)

# Create Binary Matrix

In [42]:
binary_matrix = mf.createBinaryMatix(df_interactions)

In [43]:
binary_matrix.head()

,,Smad4-Smad2-Smad3 complex,SERPINA1-CTSG complex,Prnp homodimer complex,Cpt1a-Acsl1-Vdac1 complex,CENP-T-W-S-X heterotetramer complex,PRAME-ElonginBC ubiquitin ligase,MGAT4B-SLC35A2,Smad1-Notch1-p300-Pcaf complex,Mi2/NuRD-BCL6-MTA3 complex,...,HIF1A-OS9-EGLN1 complex,"F1F0-ATP synthase, mitochondrial",SMAD2-SMAD4-WWTR1 complex,MPP4-MPP5-CRB1 complex,CD19-Vav-PI 3-kinase,SIN3-HDAC-SAP30-ARID4 complex,Gata1-Fog1-MeCP1 complex,ITGAV-ITGB3-ADAM15 complex,GPI-GnT activity complex,MT1-G
PPP1CA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
POLD1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GSE1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RBMX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZCCHC8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
binary_matrix.shape

(3753, 2752)

# Save Binary Matrix

In [45]:
filename = '~/./Documents/Harmonizome/CORUM/Output/corum_binary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
binary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Set Library

In [46]:
path = '/Users/moshesilverstein/Documents/Harmonizome/CORUM/Output/'

In [47]:
name = 'corum_gene_set'

In [48]:
mf.createUpGeneSetLib(binary_matrix, path, name)

# Create Attribute Library

In [49]:
path = '/Users/moshesilverstein/Documents/Harmonizome/CORUM/Output/'

In [50]:
name = 'corum_attribute_set'

In [51]:
mf.createUpAttributeSetLib(binary_matrix, path, name)

# Create Gene Similarity Matrix

In [52]:
gene_similarity_matix = mf.createSimilarityMatrix(binary_matrix, 'jaccard')

In [53]:
gene_similarity_matix.head()

,PPP1CA,POLD1,GSE1,RBMX,ZCCHC8,BLM,DDX20,FOSL1,CCNK,ARHGEF1,...,AAAS,LSM1,OS9,NCK1,YWHAB,WRN,GTF2A2,RPL23,SMC1A,ELK4
PPP1CA,1.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
POLD1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GSE1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RBMX,0.2,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZCCHC8,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Save Gene Similarity Matrix

In [54]:
filename = '~/./Documents/Harmonizome/CORUM/Output/corum_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute Similarity matrix

In [55]:
attribute_similarity_matix = mf.createSimilarityMatrix(binary_matrix.T, 'jaccard')

In [56]:
attribute_similarity_matix.head()

,,Smad4-Smad2-Smad3 complex,SERPINA1-CTSG complex,Prnp homodimer complex,Cpt1a-Acsl1-Vdac1 complex,CENP-T-W-S-X heterotetramer complex,PRAME-ElonginBC ubiquitin ligase,MGAT4B-SLC35A2,Smad1-Notch1-p300-Pcaf complex,Mi2/NuRD-BCL6-MTA3 complex,...,HIF1A-OS9-EGLN1 complex,"F1F0-ATP synthase, mitochondrial",SMAD2-SMAD4-WWTR1 complex,MPP4-MPP5-CRB1 complex,CD19-Vav-PI 3-kinase,SIN3-HDAC-SAP30-ARID4 complex,Gata1-Fog1-MeCP1 complex,ITGAV-ITGB3-ADAM15 complex,GPI-GnT activity complex,MT1-G
,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Smad4-Smad2-Smad3 complex,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SERPINA1-CTSG complex,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Prnp homodimer complex,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cpt1a-Acsl1-Vdac1 complex,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Save Attribute Similarity Matrix

In [57]:
filename = '~/./Documents/Harmonizome/CORUM/Output/corum_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [58]:
gene_list = mf.createGeneList(binary_matrix)

In [59]:
gene_list.head()

,GeneSym,GeneID
0,PPP1CA,5499
1,POLD1,5424
2,GSE1,23199
3,RBMX,27316
4,ZCCHC8,55596


In [60]:
gene_list.shape

(3753, 2)

# Save Gene List

In [61]:
filename = '~/./Documents/Harmonizome/CORUM/Output/corum_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute List

In [62]:
attribute_list = mf.createAttributeList(binary_matrix)

In [63]:
attribute_list.head()

,Attributes
0,
1,Smad4-Smad2-Smad3 complex
2,SERPINA1-CTSG complex
3,Prnp homodimer complex
4,Cpt1a-Acsl1-Vdac1 complex


In [64]:
attribute_list.shape

(2752, 1)

# Save Attribute List

In [65]:
filename = '~/./Documents/Harmonizome/CORUM/Output/corum_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Gene-Attribute Edge List¶

In [66]:
path = '/Users/moshesilverstein/Documents/Harmonizome/CORUM/Output/'

In [67]:
name = 'corum_gene_attribute_edge_list'

In [68]:
mf.createGeneAttributeEdgeList(binary_matrix, gene_list, path, name)

Progeres: 100%  2752 Out of 2752   

 The number of statisticaly relevent gene-attribute associations is: 11280
